In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import math
import time
import keras
import sklearn
import warnings
warnings.filterwarnings('ignore')

from keras import models, layers
from keras.layers import Dense, Activation, PReLU
from keras.regularizers import l2, l1
from keras.optimizers import Adam, RMSprop
from keras.models import Model

In [2]:
# Load the dataset
train = pd.read_csv('train.csv')

test = pd.read_csv('test.csv')

sample_sub = pd.read_csv('SampleSubmission.csv')


In [3]:
train.columns

Index(['Item_ID', 'Store_ID', 'Item_Store_ID', 'Item_Weight',
       'Item_Sugar_Content', 'Item_Visibility', 'Item_Type', 'Item_Price',
       'Store_Start_Year', 'Store_Size', 'Store_Location_Type', 'Store_Type',
       'Item_Store_Returns'],
      dtype='object')

In [4]:
train.head()

,Item_ID,Store_ID,Item_Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Returns
0,DRA12,BABATUNJI010,DRA12_BABATUNJI010,11.6,Low Sugar,0.068535,Soft Drinks,357.54,2005,NaN,Cluster 3,Grocery Store,709.08
1,DRA12,BABATUNJI013,DRA12_BABATUNJI013,11.6,Low Sugar,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,Supermarket Type1,6381.69
2,DRA12,BABATUNJI017,DRA12_BABATUNJI017,11.6,Low Sugar,0.041178,Soft Drinks,350.79,2014,NaN,Cluster 2,Supermarket Type1,6381.69
3,DRA12,BABATUNJI018,DRA12_BABATUNJI018,11.6,Low Sugar,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,Supermarket Type2,2127.23
4,DRA12,BABATUNJI035,DRA12_BABATUNJI035,11.6,Ultra Low Sugar,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,Supermarket Type1,2481.77


In [5]:
test.head()

,Item_ID,Store_ID,Item_Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type
0,DRA59,BABATUNJI010,DRA59_BABATUNJI010,8.270,Normal Sugar,0.214125,Soft Drinks,459.98,2005,NaN,Cluster 3,Grocery Store
1,DRA59,BABATUNJI013,DRA59_BABATUNJI013,8.270,Normal Sugar,0.127821,Soft Drinks,464.98,1994,High,Cluster 3,Supermarket Type1
2,DRB01,BABATUNJI013,DRB01_BABATUNJI013,7.390,Low Sugar,0.082171,Soft Drinks,477.38,1994,High,Cluster 3,Supermarket Type1
3,DRB13,BABATUNJI010,DRB13_BABATUNJI010,6.115,Normal Sugar,0.011791,Soft Drinks,472.63,2005,NaN,Cluster 3,Grocery Store
4,DRB13,BABATUNJI013,DRB13_BABATUNJI013,6.115,Normal Sugar,0.007038,Soft Drinks,473.13,1994,High,Cluster 3,Supermarket Type1


### FEATURE ENGINEERING

In [6]:
# Drop useless columns
to_drop = ['Item_ID', 'Store_ID']
train.drop(columns=to_drop, inplace=True, axis=1)
test.drop(columns=to_drop, inplace=True, axis=1)

In [7]:
train.set_index('Item_Store_ID', inplace=True)
test.set_index('Item_Store_ID', inplace=True)

In [8]:
train.head()

,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Returns
Item_Store_ID,,,,,,,,,,
DRA12_BABATUNJI010,11.6,Low Sugar,0.068535,Soft Drinks,357.54,2005,NaN,Cluster 3,Grocery Store,709.08
DRA12_BABATUNJI013,11.6,Low Sugar,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,Supermarket Type1,6381.69
DRA12_BABATUNJI017,11.6,Low Sugar,0.041178,Soft Drinks,350.79,2014,NaN,Cluster 2,Supermarket Type1,6381.69
DRA12_BABATUNJI018,11.6,Low Sugar,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,Supermarket Type2,2127.23
DRA12_BABATUNJI035,11.6,Ultra Low Sugar,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,Supermarket Type1,2481.77


In [9]:
train.isnull().sum()

Item_Weight             802
Item_Sugar_Content        0
Item_Visibility           0
Item_Type                 0
Item_Price                0
Store_Start_Year          0
Store_Size             1450
Store_Location_Type       0
Store_Type                0
Item_Store_Returns        0
dtype: int64

#### Item Weight

In [10]:
train['Item_Weight'].describe()
    
def missing_mean(df, name):
    mean = df[name].mean()
    df[name] = df[name].fillna(mean)
    return df
train = missing_mean(train, 'Item_Weight')
test = missing_mean(test, 'Item_Weight')

#### Item_Sugar_Content

In [11]:
train['Item_Sugar_Content'].describe()

count          4990
unique            3
top       Low Sugar
freq           3039
Name: Item_Sugar_Content, dtype: object

In [12]:
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
            dummy_name = f"{name}-{x}"
            df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)
encode_text_dummy(train, 'Item_Sugar_Content')
encode_text_dummy(test, 'Item_Sugar_Content')

#### Item_Visibility 	

In [13]:
train['Item_Visibility'].isnull().sum()

0

#### Item_Type

In [14]:
train['Item_Type'].value_counts()

Snack Foods              758
Fruits and Vegetables    747
Household                567
Frozen Foods             457
Canned                   376
Dairy                    350
Baking Goods             344
Health and Hygiene       307
Meat                     264
Soft Drinks              261
Breads                   137
Hard Drinks              134
Others                   100
Starchy Foods             81
Breakfast                 66
Seafood                   41
Name: Item_Type, dtype: int64

In [15]:
encode_text_dummy(train, 'Item_Type')
encode_text_dummy(test, 'Item_Type')

#### Store_Start_Year

In [16]:
train['Years'] = 2020-train['Store_Start_Year']
test['Years'] = 2020-test['Store_Start_Year']

In [17]:
train.drop('Store_Start_Year', axis=1, inplace=True)
test.drop('Store_Start_Year', axis=1, inplace=True)

#### Store_Size

In [18]:
train['Store_Size'].value_counts()

Medium    1582
Small     1364
High       594
Name: Store_Size, dtype: int64

In [19]:
train['Store_Size'] = train['Store_Size'].fillna('High')
test['Store_Size'] = test['Store_Size'].fillna('High')

In [20]:
encode_text_dummy(train, 'Store_Size')
encode_text_dummy(test, 'Store_Size')

In [21]:
train.head()

,Item_Weight,Item_Visibility,Item_Price,Store_Location_Type,Store_Type,Item_Store_Returns,Item_Sugar_Content-Low Sugar,Item_Sugar_Content-Normal Sugar,Item_Sugar_Content-Ultra Low Sugar,Item_Type-Baking Goods,...,Item_Type-Meat,Item_Type-Others,Item_Type-Seafood,Item_Type-Snack Foods,Item_Type-Soft Drinks,Item_Type-Starchy Foods,Years,Store_Size-High,Store_Size-Medium,Store_Size-Small
Item_Store_ID,,,,,,,,,,,,,,,,,,,,,
DRA12_BABATUNJI010,11.6,0.068535,357.54,Cluster 3,Grocery Store,709.08,1,0,0,0,...,0,0,0,0,1,0,15,1,0,0
DRA12_BABATUNJI013,11.6,0.040912,355.79,Cluster 3,Supermarket Type1,6381.69,1,0,0,0,...,0,0,0,0,1,0,26,1,0,0
DRA12_BABATUNJI017,11.6,0.041178,350.79,Cluster 2,Supermarket Type1,6381.69,1,0,0,0,...,0,0,0,0,1,0,6,1,0,0
DRA12_BABATUNJI018,11.6,0.041113,355.04,Cluster 3,Supermarket Type2,2127.23,1,0,0,0,...,0,0,0,0,1,0,4,0,1,0
DRA12_BABATUNJI035,11.6,0.000000,354.79,Cluster 2,Supermarket Type1,2481.77,0,0,1,0,...,0,0,0,0,1,0,9,0,0,1


#### Store_Location_Type

In [22]:
train['Store_Location_Type'].value_counts()

Cluster 3    1940
Cluster 2    1581
Cluster 1    1469
Name: Store_Location_Type, dtype: int64

In [23]:
encode_text_dummy(train, 'Store_Location_Type')
encode_text_dummy(test, 'Store_Location_Type')

#### Store_Type

In [24]:
encode_text_dummy(train, 'Store_Type')
encode_text_dummy(test, 'Store_Type')

In [25]:
train.head()

,Item_Weight,Item_Visibility,Item_Price,Item_Store_Returns,Item_Sugar_Content-Low Sugar,Item_Sugar_Content-Normal Sugar,Item_Sugar_Content-Ultra Low Sugar,Item_Type-Baking Goods,Item_Type-Breads,Item_Type-Breakfast,...,Store_Size-High,Store_Size-Medium,Store_Size-Small,Store_Location_Type-Cluster 1,Store_Location_Type-Cluster 2,Store_Location_Type-Cluster 3,Store_Type-Grocery Store,Store_Type-Supermarket Type1,Store_Type-Supermarket Type2,Store_Type-Supermarket Type3
Item_Store_ID,,,,,,,,,,,,,,,,,,,,,
DRA12_BABATUNJI010,11.6,0.068535,357.54,709.08,1,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
DRA12_BABATUNJI013,11.6,0.040912,355.79,6381.69,1,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
DRA12_BABATUNJI017,11.6,0.041178,350.79,6381.69,1,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0
DRA12_BABATUNJI018,11.6,0.041113,355.04,2127.23,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
DRA12_BABATUNJI035,11.6,0.000000,354.79,2481.77,0,0,1,0,0,0,...,0,0,1,0,1,0,0,1,0,0


In [26]:
### Item_Price
train['Item_Price'] = np.log1p(train['Item_Price'])
test['Item_Price'] = np.log1p(test['Item_Price'])

In [27]:
train

,Item_Weight,Item_Visibility,Item_Price,Item_Store_Returns,Item_Sugar_Content-Low Sugar,Item_Sugar_Content-Normal Sugar,Item_Sugar_Content-Ultra Low Sugar,Item_Type-Baking Goods,Item_Type-Breads,Item_Type-Breakfast,...,Store_Size-High,Store_Size-Medium,Store_Size-Small,Store_Location_Type-Cluster 1,Store_Location_Type-Cluster 2,Store_Location_Type-Cluster 3,Store_Type-Grocery Store,Store_Type-Supermarket Type1,Store_Type-Supermarket Type2,Store_Type-Supermarket Type3
Item_Store_ID,,,,,,,,,,,,,,,,,,,,,
DRA12_BABATUNJI010,11.600000,0.068535,5.882040,709.08,1,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
DRA12_BABATUNJI013,11.600000,0.040912,5.877147,6381.69,1,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
DRA12_BABATUNJI017,11.600000,0.041178,5.863034,6381.69,1,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0
DRA12_BABATUNJI018,11.600000,0.041113,5.875043,2127.23,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
DRA12_BABATUNJI035,11.600000,0.000000,5.874341,2481.77,0,0,1,0,0,0,...,0,0,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCZ54_BABATUNJI019,12.908838,0.145952,5.999904,406.14,1,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
NCZ54_BABATUNJI027,12.908838,0.082956,6.018934,13808.69,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
NCZ54_BABATUNJI045,14.650000,0.083528,6.009157,5685.93,1,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0


In [28]:
train_copy = train.copy()

In [29]:
y = train['Item_Store_Returns'].values
train.drop('Item_Store_Returns', axis=1, inplace=True)
X = train.values

In [30]:
X.shape

(4990, 33)

In [31]:
test_set = test.values

### Modelling

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
test_scaled = scaler.transform(test_set)

In [33]:
from sklearn.model_selection import KFold, ShuffleSplit
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

In [39]:
param_grid = {'learning_rate':[0.1,0.01,0.001], 'depth':[5,10, 20], 'n_estimators':[10, 20, 30]}

In [40]:
cat = CatBoostRegressor()
n_fold = 50
kfold = KFold(n_splits=n_fold)
grid = GridSearchCV(CatBoostRegressor(), param_grid, cv=kfold)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
grid.fit(X_train, y_train)

0:	learn: 4199.9918910	total: 266ms	remaining: 2.39s
1:	learn: 4013.8001070	total: 271ms	remaining: 1.08s
2:	learn: 3840.1278875	total: 276ms	remaining: 644ms
3:	learn: 3692.7068666	total: 281ms	remaining: 421ms
4:	learn: 3569.4471253	total: 286ms	remaining: 286ms
5:	learn: 3461.1440358	total: 289ms	remaining: 193ms
6:	learn: 3372.6545318	total: 292ms	remaining: 125ms
7:	learn: 3292.4086293	total: 295ms	remaining: 73.7ms
8:	learn: 3237.2491271	total: 298ms	remaining: 33.1ms
9:	learn: 3180.3582601	total: 301ms	remaining: 0us
0:	learn: 4217.3562512	total: 5.15ms	remaining: 46.4ms
1:	learn: 4029.5363756	total: 10.5ms	remaining: 42.1ms
2:	learn: 3854.4438015	total: 14.5ms	remaining: 33.8ms
3:	learn: 3706.3679129	total: 18.1ms	remaining: 27.1ms
4:	learn: 3582.2151079	total: 21.3ms	remaining: 21.3ms
5:	learn: 3473.1564762	total: 24.3ms	remaining: 16.2ms
6:	learn: 3387.3128230	total: 27.7ms	remaining: 11.9ms
7:	learn: 3306.1689434	total: 30.4ms	remaining: 7.59ms
8:	learn: 3247.0898911	total: 

0:	learn: 4199.7672798	total: 6.48ms	remaining: 58.3ms
1:	learn: 4013.6729290	total: 11.7ms	remaining: 46.9ms
2:	learn: 3839.8961075	total: 16.2ms	remaining: 37.7ms
3:	learn: 3698.3207711	total: 18.8ms	remaining: 28.2ms
4:	learn: 3574.2944765	total: 21.5ms	remaining: 21.5ms
5:	learn: 3465.0910694	total: 24.2ms	remaining: 16.1ms
6:	learn: 3380.1092127	total: 26.8ms	remaining: 11.5ms
7:	learn: 3298.9151277	total: 29.5ms	remaining: 7.39ms
8:	learn: 3240.4771539	total: 32.4ms	remaining: 3.6ms
9:	learn: 3182.5324690	total: 35.1ms	remaining: 0us
0:	learn: 4207.9194890	total: 8.47ms	remaining: 76.2ms
1:	learn: 4020.7482935	total: 13.7ms	remaining: 54.9ms
2:	learn: 3845.8705396	total: 19ms	remaining: 44.3ms
3:	learn: 3698.3769191	total: 25.8ms	remaining: 38.8ms
4:	learn: 3574.5721504	total: 29.4ms	remaining: 29.4ms
5:	learn: 3465.5226501	total: 31.4ms	remaining: 20.9ms
6:	learn: 3375.6690569	total: 34.3ms	remaining: 14.7ms
7:	learn: 3297.1821543	total: 36.3ms	remaining: 9.08ms
8:	learn: 3239.3

0:	learn: 4205.0684810	total: 5.99ms	remaining: 53.9ms
1:	learn: 4020.3419180	total: 11.3ms	remaining: 45.2ms
2:	learn: 3857.3948297	total: 16ms	remaining: 37.3ms
3:	learn: 3716.0672670	total: 18.7ms	remaining: 28.1ms
4:	learn: 3593.3293075	total: 21.1ms	remaining: 21.1ms
5:	learn: 3483.6133509	total: 23.5ms	remaining: 15.7ms
6:	learn: 3398.3919075	total: 25.8ms	remaining: 11.1ms
7:	learn: 3318.5652614	total: 28.5ms	remaining: 7.11ms
8:	learn: 3259.7086046	total: 30.9ms	remaining: 3.43ms
9:	learn: 3200.9645661	total: 33.3ms	remaining: 0us
0:	learn: 4213.9961725	total: 5.14ms	remaining: 46.3ms
1:	learn: 4026.1977275	total: 9.93ms	remaining: 39.7ms
2:	learn: 3850.9201808	total: 13ms	remaining: 30.3ms
3:	learn: 3708.5815381	total: 16ms	remaining: 24.1ms
4:	learn: 3576.5067500	total: 19.1ms	remaining: 19.1ms
5:	learn: 3466.5875748	total: 21.7ms	remaining: 14.4ms
6:	learn: 3381.5855788	total: 24ms	remaining: 10.3ms
7:	learn: 3301.1444367	total: 26.5ms	remaining: 6.63ms
8:	learn: 3245.506894

0:	learn: 4220.6424059	total: 11.9ms	remaining: 107ms
1:	learn: 4032.9075696	total: 17.7ms	remaining: 70.7ms
2:	learn: 3857.5379600	total: 21.2ms	remaining: 49.5ms
3:	learn: 3711.6107581	total: 24ms	remaining: 36ms
4:	learn: 3586.9598049	total: 26.2ms	remaining: 26.2ms
5:	learn: 3476.7266163	total: 28.2ms	remaining: 18.8ms
6:	learn: 3386.3322380	total: 30.9ms	remaining: 13.2ms
7:	learn: 3306.9662398	total: 32.8ms	remaining: 8.21ms
8:	learn: 3248.7982253	total: 34.5ms	remaining: 3.83ms
9:	learn: 3191.0434985	total: 36.4ms	remaining: 0us
0:	learn: 4190.8290502	total: 6.43ms	remaining: 57.9ms
1:	learn: 4004.2640385	total: 11.9ms	remaining: 47.6ms
2:	learn: 3840.3967991	total: 17.6ms	remaining: 41ms
3:	learn: 3698.2872383	total: 22.9ms	remaining: 34.3ms
4:	learn: 3573.0587911	total: 28.1ms	remaining: 28.1ms
5:	learn: 3462.7940708	total: 33.1ms	remaining: 22.1ms
6:	learn: 3373.6712595	total: 35.5ms	remaining: 15.2ms
7:	learn: 3292.7157185	total: 37.7ms	remaining: 9.43ms
8:	learn: 3236.70481

0:	learn: 4218.8340844	total: 5.42ms	remaining: 103ms
1:	learn: 4030.9083521	total: 8.63ms	remaining: 77.7ms
2:	learn: 3855.2131215	total: 11.3ms	remaining: 63.8ms
3:	learn: 3707.6486696	total: 13.8ms	remaining: 55.2ms
4:	learn: 3583.6925596	total: 17.1ms	remaining: 51.4ms
5:	learn: 3474.2859988	total: 19.5ms	remaining: 45.6ms
6:	learn: 3383.8326283	total: 21.9ms	remaining: 40.7ms
7:	learn: 3301.9428060	total: 24.6ms	remaining: 36.9ms
8:	learn: 3245.3005161	total: 26.7ms	remaining: 32.6ms
9:	learn: 3187.3847601	total: 28.8ms	remaining: 28.8ms
10:	learn: 3138.7352105	total: 31ms	remaining: 25.3ms
11:	learn: 3099.3606479	total: 34.5ms	remaining: 23ms
12:	learn: 3068.7518979	total: 36.5ms	remaining: 19.7ms
13:	learn: 3044.7698850	total: 38.7ms	remaining: 16.6ms
14:	learn: 3018.4091079	total: 43.2ms	remaining: 14.4ms
15:	learn: 2996.4924992	total: 45.1ms	remaining: 11.3ms
16:	learn: 2977.0568595	total: 47.2ms	remaining: 8.32ms
17:	learn: 2963.3379077	total: 49.4ms	remaining: 5.49ms
18:	lea

...

17:	learn: 4400.7316285	total: 178ms	remaining: 118ms
18:	learn: 4398.5097041	total: 198ms	remaining: 115ms
19:	learn: 4396.1527081	total: 203ms	remaining: 101ms
20:	learn: 4393.8474072	total: 208ms	remaining: 89.3ms
21:	learn: 4391.5186463	total: 210ms	remaining: 76.5ms
22:	learn: 4389.3093002	total: 227ms	remaining: 69.2ms
23:	learn: 4387.1139752	total: 244ms	remaining: 61.1ms
24:	learn: 4384.8664376	total: 261ms	remaining: 52.3ms
25:	learn: 4382.6322909	total: 264ms	remaining: 40.6ms
26:	learn: 4380.3657872	total: 267ms	remaining: 29.7ms
27:	learn: 4378.0487889	total: 284ms	remaining: 20.3ms
28:	learn: 4375.9375177	total: 301ms	remaining: 10.4ms
29:	learn: 4373.6700359	total: 303ms	remaining: 0us
0:	learn: 4429.9163957	total: 7.26ms	remaining: 211ms
1:	learn: 4427.6281137	total: 35.8ms	remaining: 502ms
2:	learn: 4425.3065478	total: 39.7ms	remaining: 357ms
3:	learn: 4423.0467869	total: 43.2ms	remaining: 281ms
4:	learn: 4420.7443049	total: 62.2ms	remaining: 311ms
5:	learn: 4418.423557

0:	learn: 4443.5761837	total: 10.2ms	remaining: 297ms
1:	learn: 4441.2614495	total: 39.9ms	remaining: 559ms
2:	learn: 4438.9176812	total: 42.7ms	remaining: 385ms
3:	learn: 4436.6417597	total: 45.7ms	remaining: 297ms
4:	learn: 4434.3138198	total: 66.2ms	remaining: 331ms
5:	learn: 4431.9718857	total: 72.6ms	remaining: 290ms
6:	learn: 4429.6318618	total: 78.4ms	remaining: 258ms
7:	learn: 4427.2669250	total: 90.1ms	remaining: 248ms
8:	learn: 4425.0464482	total: 94.9ms	remaining: 221ms
9:	learn: 4422.6890216	total: 128ms	remaining: 256ms
10:	learn: 4420.4659073	total: 158ms	remaining: 273ms
11:	learn: 4418.2492777	total: 163ms	remaining: 245ms
12:	learn: 4415.8497697	total: 173ms	remaining: 226ms
13:	learn: 4413.5211909	total: 179ms	remaining: 204ms
14:	learn: 4411.3122172	total: 208ms	remaining: 208ms
15:	learn: 4409.0013982	total: 211ms	remaining: 185ms
16:	learn: 4406.7130072	total: 231ms	remaining: 176ms
17:	learn: 4404.4520406	total: 232ms	remaining: 155ms
18:	learn: 4402.0864726	total

16:	learn: 4371.2433739	total: 219ms	remaining: 168ms
17:	learn: 4369.0162583	total: 238ms	remaining: 159ms
18:	learn: 4366.8191851	total: 240ms	remaining: 139ms
19:	learn: 4364.6180462	total: 257ms	remaining: 128ms
20:	learn: 4362.4916411	total: 275ms	remaining: 118ms
21:	learn: 4360.2000926	total: 280ms	remaining: 102ms
22:	learn: 4357.9330156	total: 297ms	remaining: 90.5ms
23:	learn: 4355.6812501	total: 315ms	remaining: 78.7ms
24:	learn: 4353.5606624	total: 332ms	remaining: 66.5ms
25:	learn: 4351.2776953	total: 341ms	remaining: 52.5ms
26:	learn: 4349.1008467	total: 359ms	remaining: 39.8ms
27:	learn: 4347.0193982	total: 360ms	remaining: 25.7ms
28:	learn: 4344.7929804	total: 377ms	remaining: 13ms
29:	learn: 4342.6615858	total: 394ms	remaining: 0us
0:	learn: 4440.1078596	total: 12.2ms	remaining: 354ms
1:	learn: 4437.7957860	total: 40ms	remaining: 560ms
2:	learn: 4435.4445438	total: 43.7ms	remaining: 394ms
3:	learn: 4433.1576764	total: 46.8ms	remaining: 304ms
4:	learn: 4430.8286460	tota

28:	learn: 4364.9751232	total: 374ms	remaining: 12.9ms
29:	learn: 4362.6638982	total: 392ms	remaining: 0us
0:	learn: 4448.0904508	total: 7.24ms	remaining: 210ms
1:	learn: 4445.7730827	total: 36.8ms	remaining: 515ms
2:	learn: 4443.4178702	total: 41.5ms	remaining: 373ms
3:	learn: 4441.1220530	total: 44.4ms	remaining: 289ms
4:	learn: 4438.8029911	total: 61.9ms	remaining: 310ms
5:	learn: 4436.4599672	total: 71.3ms	remaining: 285ms
6:	learn: 4434.1358949	total: 80.3ms	remaining: 264ms
7:	learn: 4432.0304305	total: 97.4ms	remaining: 268ms
8:	learn: 4429.6992956	total: 107ms	remaining: 249ms
9:	learn: 4427.4178153	total: 124ms	remaining: 248ms
10:	learn: 4425.1092868	total: 133ms	remaining: 230ms
11:	learn: 4422.8229169	total: 153ms	remaining: 229ms
12:	learn: 4420.5064488	total: 158ms	remaining: 207ms
13:	learn: 4418.1672899	total: 167ms	remaining: 191ms
14:	learn: 4415.9617943	total: 184ms	remaining: 184ms
15:	learn: 4413.7440638	total: 195ms	remaining: 170ms
16:	learn: 4411.4750057	total: 

GridSearchCV(cv=KFold(n_splits=50, random_state=None, shuffle=False),
             error_score=nan,
             estimator=<catboost.core.CatBoostRegressor object at 0x000001D71601CB88>,
             iid='deprecated', n_jobs=None,
             param_grid={'depth': [5, 10, 20],
                         'learning_rate': [0.1, 0.01, 0.001],
                         'n_estimators': [10, 20, 30]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [41]:
print(grid.score(X_test, y_test))
print(grid.best_params_)
print(grid.best_score_)

0.5524589891652525
{'depth': 5, 'learning_rate': 0.1, 'n_estimators': 30}
0.5561372985506723


In [31]:
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, ShuffleSplit
SPLITS = 50

boot = KFold(n_splits=10, shuffle=True, random_state=42)
oos_pred = []
oos_y = []
fold = 0
# Track progress
for train, test in boot.split(X_scaled):
    fold+=1
    print('Fold #',fold)
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
    
    cat = CatBoostRegressor(depth=16, learning_rate=0.1, n_estimators=1000)
    cat.fit(X_train, y_train, early_stopping_rounds = 100,eval_set=(X_test, y_test), use_best_model=True)
    pred = cat.predict(X_test)
    oos_y.append(y_test)
    oos_pred.append(pred)
    
    score = np.sqrt(mean_squared_error(pred, y_test))
    print('Fold score (RMSE):', score)
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)

score = np.sqrt(mean_squared_error(oos_pred, oos_y))
print(f"Final, out of sample score (RMSE): {score}")    


Fold # 1


NameError: name 'CatBoostRegressor' is not defined

In [72]:
# Submission
predictions = cat.predict(test_set)
sub = pd.DataFrame()
sub['Item_Store_ID'] = sample_sub['Item_Store_ID']
sub['Item_Store_Returns'] = predictions
sub.to_csv('answer1.csv', index=False)

**You can improve your score with some of these techniques:**

1) Try more feature engineering techniques

2) Try Feature Selection techniques like Univariate Feature Selection, Recursive Feature Elimination, Model-Based Feature Selection.

3) Try using another model: LGBRegressor, XGBRegressor, etc

4) Try tuning more hyperparamters 
